# Data Loading, Storage, and File Formats
---
DAT 512 Canisuis College <br>
Professor Paul Lambson<br>
<br>
### Learning Objectives
- Become familiar with ways to load data into pandas
- Become confident with loading simple data
- Learn about multiple datatypes
- Learn how to write data from pandas to files
- Learn a simple method for access data from APIs
- Learn how to create a lightweigh database
<br>


### Sections
- [Reading and Writing Data in Text Format](#reading-and-writing-data-in-text-format)
- [Reading Text Files in Pieces](#reading-text-files-in-pieces)
- [Writing Data to Text Format](#writing-data-to-text-format)
- [Working With Other Delimited Formats](#working-with-other-delimited-formats)
- [JSON Data](#json-data)
- [HTML Web Scraping](#html-web-scraping)
- [Binary Data Formats](#binary-data-formats)
- [Reading Microsoft Excel Formats](#reading-microsoft-excel-files)
- [Interacting with Databases](#interacting-with-databases)

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)

<a id='reading-and-writing-data-in-text-format'></a>
# Reading and Writing Data in Text Format

In [ ]:
# look at a text file
!cat examples/ex1.csv

In [ ]:
# read in text file, look at pd.read_ options
df = pd.read_csv("examples/ex1.csv")
df

In [ ]:
#! ipython id=04eaafad5e034dd888ca309bfa6fd75c
!cat examples/ex2.csv

In [ ]:
# input file has no headers
pd.read_csv("examples/ex2.csv", header=None)

In [ ]:
# names keyword can pass a list of column names
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

In [ ]:
# can use a named list to put into names and identify the index
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

In [ ]:
# mutliindex is a possiblilty
!cat examples/csv_mindex.csv
parsed = pd.read_csv("examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

In [ ]:
# consider an irregularly delimitted file
!cat examples/ex3.txt

In [ ]:
# can use regex to parse
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

In [ ]:
# can skip rows if needed
!cat examples/ex4.csv
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

In [ ]:
# Null values shown as NaN
!cat examples/ex5.csv
result = pd.read_csv("examples/ex5.csv")
result

In [ ]:
# use isna to find missing data
pd.isna(result)

In [ ]:
# can add to the list of items that are recognized as NaN
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])
result

In [ ]:
# not keeping default na will break .isna()
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)
result2

In [ ]:
result2.isna()

In [ ]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False,
                      na_values=["NA"])
result3

In [ ]:
result3.isna()

In [ ]:
# Different NA sentinels can be specified for each column in a dictionary
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels,

            keep_default_na=False)

<a id='reading-text-files-in-pieces'></a>
# Reading Text Files in Pieces

In [ ]:
# decrease display for larger files
pd.options.display.max_rows = 10

In [ ]:
# note the elipsis '...' that indicated data that is not shown
result = pd.read_csv("examples/ex6.csv")
result

In [ ]:
# could read in only a smalle amount of rows
pd.read_csv("examples/ex6.csv", nrows=5)

In [ ]:
# to read in a file in peices a chunksize can be specified
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

In [ ]:
# iterate over a file to add key.value_counts() to a series
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

In [ ]:
# result of chunking
tot[:10]

<a id='writing-data-to-text-format'></a>
# Writing Data to Text Format

In [ ]:
# lead a dataframe to write
data = pd.read_csv("examples/ex5.csv")
data

In [ ]:
# use to_csv(), look at other to options
data.to_csv("examples/out.csv")
!cat examples/out.csv

In [ ]:
# Other delimiters can be used, of course 
# (writing to sys.stdout so it prints the text result to the console rather than a file)
import sys
data.to_csv(sys.stdout, sep="|")

In [ ]:
# write nulls as a custom representation
data.to_csv(sys.stdout, na_rep="NULL")

In [ ]:
# dropping index and headers
data.to_csv(sys.stdout, index=False, header=False)

In [ ]:
# dropping index, overwriting column names
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

<a id='working-with-other-delimited-formats'></a>
# Working with Other Delimited Formats

In [ ]:
# if a file is more complicated than read_csv() can handle, move into csv library
!cat examples/ex7.csv

In [ ]:
# example of using csv library
import csv
f = open("examples/ex7.csv")
reader = csv.reader(f)

In [ ]:
# csv readers iterate over a file, this is the core ETL process
for line in reader:
    print(line)
f.close()

In [ ]:
# if doing the wrangling then move through this list of steps
# list of lines
with open("examples/ex7.csv") as f:
    lines = list(csv.reader(f))

In [ ]:
# first line is headers, following lines are values
header, values = lines[0], lines[1:]

In [ ]:
# create a dictionary that has headers and zipped values, then load to data frame
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

<a id='json-data'></a>
# JSON Data

In [ ]:
# Begin with an string that is a json
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [ ]:
#use python json library to convert to actual json
import json
result = json.loads(obj)
result

In [ ]:
# can be turned back into a string
asjson = json.dumps(result)
asjson

In [ ]:
# transverse a JSON in order to load a child dictionary
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

In [ ]:
#
!cat examples/example.json

In [ ]:
# pandas has a read_json() option for relatively uniform JSON objects
data = pd.read_json("examples/example.json")
data

In [ ]:
# options exist to write a file
data.to_json(sys.stdout)

In [ ]:
# columns or row first
data.to_json(sys.stdout, orient="records")

<a id='html-web-scraping'></a>
# HTML Web Scraping

In [ ]:
# read a html file that holds a table
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)

In [ ]:
failures = tables[0]
failures.head()

In [ ]:
# convert a date field to timestamp
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

<a id='binary-data-formats'></a>
# Binary Data Formats

In [ ]:
# One simple way to store (or serialize) data in binary format is using Python’s built-in pickle module
frame = pd.read_csv("examples/ex1.csv")
frame

In [ ]:
frame.to_pickle("examples/frame_pickle")

In [ ]:
# read back pickle
pd.read_pickle("examples/frame_pickle")

In [ ]:
# delete the example file
!rm examples/frame_pickle

In [ ]:
!pip install pyarrow

In [ ]:
# support of Apache Parquet
fec = pd.read_parquet('examples/fec.parquet')

<a id='reading-microsoft-excel-files'></a>
# Reading Microsoft Excel Files

In [ ]:
# !pip install openpyxl xlrd

In [ ]:
# load a excel file
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [ ]:
# look at sheet names
xlsx.sheet_names

In [ ]:
# parsed a sheet into a data frame
xlsx.parse(sheet_name="Sheet1")

In [ ]:
# indicate an index col
xlsx.parse(sheet_name="Sheet1", index_col=0)

In [ ]:
# if you are just reading in one sheet, pass that value
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

In [ ]:
# writing excel require initialization of a writer
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.save()

In [ ]:
# can also to_excel() which will 
frame.to_excel("examples/ex2.xlsx")

In [ ]:
# delete files
!rm examples/ex2.xlsx

In [ ]:
# To find the last 30 GitHub issues for pandas on GitHub
# we can make a GET HTTP request using the add-on requests library
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

In [ ]:
# turn the response into json and view first 
data = resp.json()
data[0]["title"]

In [ ]:
# pass the elements to load into the data frame
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

<a id='interacting-with-databases'></a>
# Interacting with Databases

In [ ]:
# create a sqlite database 
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)
con.commit()

In [ ]:
# load the database using SQL statement
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [ ]:
# pull data from database using sql query
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

In [ ]:
# use pandas to pull from the cursor
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

In [ ]:
# use sqlalchemy as an alternative to read with engine, rather than cursor
import sqlalchemy as sqla
db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

In [ ]:
# delete database
!rm mydata.sqlite

<a id='in-class-problems'></a>
# In Class Problems

In [ ]:
'''
    #1
    Create a dataframe of the file called 'tips' from the examples folder
    How many rows and columns?
    Look at the first 5 rows and last 5 rows?
    Print the column names
    Print the column datatypes
    What is the independent variables?
'''
'''
    #2
    Calculate summary statistics
    What percent of observations were smokers?
    What is the distribution of days of week?
    What is the distribution of time?
    What is the average size?
    What is the averae percentage tip?
'''
'''
    #3
    Write the table to a json that is a list of dictionaries where the list is 244 long
    Create a sqlite database and load the dataframe to the database
'''
'''
    #4
    Build a correlation matrix
    Create a model that predicts tip amount
'''